In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Timezone,Date
0,0,Alfenas,-21.4292,-45.9472,25.82,63,99,2.32,BR,-10800,1697382706
1,1,Iqaluit,63.7506,-68.5145,0.85,86,100,3.60,CA,-14400,1697382707
2,2,Saint-Pierre,-21.3393,55.4781,25.82,73,0,2.57,RE,14400,1697382707
3,3,Tiksi,71.6872,128.8694,-10.05,90,100,5.69,RU,32400,1697382707
4,4,Kashgar,39.4547,75.9797,17.75,19,77,1.59,CN,28800,1697382708


Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.¶

In [3]:
# Configure the map
Humi_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Max Temp"]
)



# Display the map plot
Humi_map

C:\Users\Sch\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp)

Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

# Clean Data for the next steps

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Example df  had droped all max temps below 20C
good = city_data_df.loc[(city_data_df['Max Temp']>=17) & (city_data_df['Max Temp']<=27)& (city_data_df['Wind Speed']<=8.12)&(city_data_df['Cloudiness']>=30)]
# example df has no tempratures above 27C
# Example df looked like it had droped wind above 4.12mph, but that shrinks the data too much


# Drop any rows with null values
C_good = good.dropna()
# Display sample data
good_weather = pd.DataFrame(C_good)
good_weather

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Timezone,Date
0,0,Alfenas,-21.4292,-45.9472,25.82,63,99,2.32,BR,-10800,1697382706
4,4,Kashgar,39.4547,75.9797,17.75,19,77,1.59,CN,28800,1697382708
9,9,East London,-33.0153,27.9116,20.31,76,96,7.96,ZA,7200,1697382709
17,17,Port Elizabeth,-33.9180,25.5701,20.06,83,53,2.24,ZA,7200,1697382620
21,21,San Julián,21.0167,-102.1667,18.76,67,99,2.53,MX,-21600,1697382713
...,...,...,...,...,...,...,...,...,...,...,...
519,519,San Felice Circeo,41.2372,13.0942,25.95,74,66,0.89,IT,7200,1697382884
525,525,Labasa,-16.4167,179.3833,23.66,89,43,3.24,FJ,43200,1697382886
529,529,Ishigaki,24.3448,124.1572,25.07,83,100,2.57,JP,32400,1697382888
531,531,Jayapura,-2.5333,140.7000,25.66,89,79,1.86,ID,32400,1697382888


In [5]:
# count my results. There were no null values in my data set as the length was the same with and without the notnull
len(good_weather)

80

# Hotel DataFrame

In [6]:
hotel_df = good_weather[['City', 'Country','Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Alfenas,BR,-21.4292,-45.9472,63,
4,Kashgar,CN,39.4547,75.9797,19,
9,East London,ZA,-33.0153,27.9116,76,
17,Port Elizabeth,ZA,-33.9180,25.5701,83,
21,San Julián,MX,21.0167,-102.1667,67,
...,...,...,...,...,...,...
519,San Felice Circeo,IT,41.2372,13.0942,74,
525,Labasa,FJ,-16.4167,179.3833,89,
529,Ishigaki,JP,24.3448,124.1572,83,
531,Jayapura,ID,-2.5333,140.7000,89,


In [7]:
# Nukmber question!!
hotel_df.count()

City          80
Country       80
Lat           80
Lng           80
Humidity      80
Hotel Name    80
dtype: int64

# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data

hotel_df

Starting hotel search
Alfenas - nearest hotel: Hotel JS
Kashgar - nearest hotel: 深航国际酒店
East London - nearest hotel: No hotel found
Port Elizabeth - nearest hotel: Waterford Hotel
San Julián - nearest hotel: No hotel found
Kruisfontein - nearest hotel: No hotel found
Lata - nearest hotel: Residencial Botânico
Margaret River - nearest hotel: Margaret River Hotel
Lihue - nearest hotel: Kauai Palms
Wailua Homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
La Passe - nearest hotel: L'Estuaire
Tsiombe - nearest hotel: No hotel found
Port Mathurin - nearest hotel: Escale Vacances
Hawaiian Paradise Park - nearest hotel: No hotel found
Camaná - nearest hotel: Caja Arequipa
Vila Velha - nearest hotel: Hotel Prainha
Treinta y Tres - nearest hotel: No hotel found
Saldanha - nearest hotel: Hoedjesbaai
Mugia - nearest hotel: Solpor do Coído
Hirara - nearest hotel: 古謝そば屋
Port Alfred - nearest hotel: No hotel found
Kapaa - nearest hotel: Pono Kai Resort
Nabire - nearest hotel: Hotel Anggr

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Alfenas,BR,-21.4292,-45.9472,63,Hotel JS
4,Kashgar,CN,39.4547,75.9797,19,深航国际酒店
9,East London,ZA,-33.0153,27.9116,76,No hotel found
17,Port Elizabeth,ZA,-33.9180,25.5701,83,Waterford Hotel
21,San Julián,MX,21.0167,-102.1667,67,No hotel found
...,...,...,...,...,...,...
519,San Felice Circeo,IT,41.2372,13.0942,74,Centro Ferie Salvatore
525,Labasa,FJ,-16.4167,179.3833,89,Grand Eastern Hotel
529,Ishigaki,JP,24.3448,124.1572,83,アートホテル石垣島
531,Jayapura,ID,-2.5333,140.7000,89,Hotel Jasmine Jayapura


In [9]:
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Alfenas,BR,-21.4292,-45.9472,63,Hotel JS
4,Kashgar,CN,39.4547,75.9797,19,深航国际酒店
9,East London,ZA,-33.0153,27.9116,76,No hotel found
17,Port Elizabeth,ZA,-33.9180,25.5701,83,Waterford Hotel
21,San Julián,MX,21.0167,-102.1667,67,No hotel found


# Clean locations without hotel

In [10]:
Has_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!= 'No hotel found']
len(Has_hotel_df)

60

# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
# Configure the map
Humi2_map = Has_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"]
)

# Display the map plot
Humi2_map

C:\Users\Sch\anaconda3\Lib\site-packages\holoviews\util\transform.py:672: RuntimeWarning: invalid value encountered in sqrt
  data = fn(*args, **kwargs)


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)